In [1]:
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup

In [2]:
# importing the csv into a dataframe, csv file should be in same directory as python script
members_csv = pd.read_csv("slack-ieee-ucb-members.csv")
members_csv

,username,email,status,billing-active,has-2fa,has-sso,userid,fullname,displayname,expiration-timestamp
0,jared,rulison@berkeley.edu,Deactivated,0,0,0,U1J8N2NSF,Jared Rulison,jared,NaN
1,vicky11z,vicky11z@berkeley.edu,Deactivated,0,0,0,U1J9V8VRC,Vicky Zhang,vicky11z,NaN
2,ieeeucb,ieeeucb@gmail.com,Primary Owner,0,0,0,U1J9ZTAJ3,IEEE Account,ieeeucb,NaN
3,kevinma,kevinma.sd@berkeley.edu,Member,0,0,0,U1JAHG117,Kevin Ma,kevinma,NaN
4,cporter,cporter15@berkeley.edu,Deactivated,0,0,0,U1KSG2XLH,Chase Porter,cporter,NaN
...,...,...,...,...,...,...,...,...,...,...
784,omairkhan,omairkhan@berkeley.edu,Member,1,0,0,U01J6DP124B,Omair Khan,Omair Khan,NaN
785,leobookey,leobookey@berkeley.edu,Member,1,0,0,U01JB3Q4FNF,Leo Bookey,Leo Bookey,NaN
786,rexliu3,rexliu3@berkeley.edu,Member,1,0,0,U01JGSCP96J,Rex Liu,Rex Liu,NaN
787,zoehinks,zoehinks@berkeley.edu,Member,1,0,0,U01JSFZ6RK5,Robin Hinks,Robin Hinks,NaN


In [3]:
# filtering out deactivated accounts
not_deactivated = members_csv['status'] != 'Deactivated'
members_csv = members_csv[not_deactivated]

# filtering out active members 
inactive = members_csv['billing-active'] != 1
members_csv = members_csv[inactive]

members_csv

,username,email,status,billing-active,has-2fa,has-sso,userid,fullname,displayname,expiration-timestamp
2,ieeeucb,ieeeucb@gmail.com,Primary Owner,0,0,0,U1J9ZTAJ3,IEEE Account,ieeeucb,NaN
3,kevinma,kevinma.sd@berkeley.edu,Member,0,0,0,U1JAHG117,Kevin Ma,kevinma,NaN
13,daminig,daminig@berkeley.edu,Member,0,0,0,U2AEXK1R9,Damini Grover,daminig,NaN
20,brent,brentyi@berkeley.edu,Admin,0,0,0,U2D1SNSUE,Brent Yi,brent,NaN
24,billz12oz,william_zhao@berkeley.edu,Member,0,0,0,U2D2AHPCZ,Billy Zhao,billz12oz,NaN
...,...,...,...,...,...,...,...,...,...,...
777,eustyn_trinh,eustyn_trinh@berkeley.edu,Member,0,0,0,U01DN805KEG,Waterstar1,Waterstar1,NaN
779,richard95lee,richard95lee@berkeley.edu,Member,0,0,0,U01E93SKUPL,Richard Lee,Richard Lee,NaN
780,scheduler,botuser-T1JAA6H2N-B01EPJZ3EHH@slack-bots.com,Bot,0,0,0,U01F220B6U9,Message Scheduler,NaN,NaN
781,xr,xr@berkeley.edu,Member,0,0,0,U01F89GEPS5,Ray Xi,Ray Xi,NaN


In [4]:
member_activity = members_csv[['username', 'email', 'status', 'fullname']].copy()
member_activity['in_directory'] = 'nan' 
member_activity

,username,email,status,fullname,in_directory
2,ieeeucb,ieeeucb@gmail.com,Primary Owner,IEEE Account,nan
3,kevinma,kevinma.sd@berkeley.edu,Member,Kevin Ma,nan
13,daminig,daminig@berkeley.edu,Member,Damini Grover,nan
20,brent,brentyi@berkeley.edu,Admin,Brent Yi,nan
24,billz12oz,william_zhao@berkeley.edu,Member,Billy Zhao,nan
...,...,...,...,...,...
777,eustyn_trinh,eustyn_trinh@berkeley.edu,Member,Waterstar1,nan
779,richard95lee,richard95lee@berkeley.edu,Member,Richard Lee,nan
780,scheduler,botuser-T1JAA6H2N-B01EPJZ3EHH@slack-bots.com,Bot,Message Scheduler,nan
781,xr,xr@berkeley.edu,Member,Ray Xi,nan


In [42]:
# an example of when the email exists in the directory
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
does_not_exist = 'No matches to your search. Please try again.\n'
test_URL_1 = "https://www.berkeley.edu/directory/results?search-term=indianjit%40berkeley.edu" 
test_page_1 = requests.get(test_URL_1, headers=headers)

test_soup_1 = BeautifulSoup(test_page_1.content, 'html.parser')
search_results_1 = test_soup_1.find_all('section', {'class': 'search-results'})

for result in search_results_1:
    print(does_not_exist in result.text)
    print(result.text)

False

Indianjit Singh
Emailindianjit@berkeley.edu
UID1718546



In [43]:
# an example of when the email does not exist in the directory 
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
does_not_exist = 'No matches to your search. Please try again.'
test_URL_2 = 'https://www.berkeley.edu/directory/results?search-term=brentyi%40berkeley.edu'
test_page_2 = requests.get(test_URL_2, headers=headers)

test_soup_2 = BeautifulSoup(test_page_2.content, 'html.parser')
search_results_2 = test_soup_2.find_all('section', {'class': 'search-results'})

for result in search_results_2:
    print(does_not_exist in result.text)
    print(result.text)

True

No matches to your search. Please try again.



In [5]:
# scraping @berkeley.edu emails from the cal directory, stalling between requests to avoid 403 forbidden 
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
does_not_exist = 'No matches to your search. Please try again.'
base_URL = "https://www.berkeley.edu/directory/results"
indices = member_activity.index

for i in range(len(member_activity)):
    member = member_activity.iloc[i]
    query = '?search-term=' + member['email'].replace('@', '%40')
    full_query = base_URL + query
    page = requests.get(full_query, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    search_results = soup.find_all('section', {'class': 'search-results'})
    
    for result in search_results:
        if does_not_exist in result.text:
            member_activity.at[indices[i], 'in_directory'] = 0
            # member_activity['in_directory', indices[i]] = 0
        else:
            member_activity.at[indices[i], 'in_directory'] = 1
            # member_activity['in_directory', indices[i]] = 1
    
    print(str(member['email']) + " " + str(does_not_exist in result.text))
    time.sleep(5)
    
member_activity

ieeeucb@gmail.com True
kevinma.sd@berkeley.edu True
daminig@berkeley.edu True
brentyi@berkeley.edu True
william_zhao@berkeley.edu True
tiffany_chien@berkeley.edu False
rliu4439@berkeley.edu True
nehagodbole@berkeley.edu True
robertquitt@berkeley.edu True
benjaminli@berkeley.edu True
alicewang96@berkeley.edu True
ahadrauf@berkeley.edu False
sumukhshiv@berkeley.edu True
cathymeng@berkeley.edu True
srividhyashanker@berkeley.edu True
kangdoreene@berkeley.edu True
raymondchong@berkeley.edu True
mayhuang321@gmail.com True
gkliger@berkeley.edu True
tbharadwaj@berkeley.edu True
henrymuller@berkeley.edu True
alexpissinoumakki@berkeley.edu False
anoosha.balebail@berkeley.edu False
sthalanki@berkeley.edu False
gbalke@berkeley.edu True
edward.fang@berkeley.edu True
jamesmarsh99@berkeley.edu False
lbhattacharjee@berkeley.edu False
alexkassil@berkeley.edu False
rayaltenberg@berkeley.edu False
abhupatiraju@berkeley.edu True
pravin.ravishanker@berkeley.edu False
wml@berkeley.edu True
hallchen@berkeley

vmadduri@berkeley.edu False
billwang2551@gmail.com True
yuxiao.huang@berkeley.edu False
ndkanakadandila@berkeley.edu False
desireewang@berkeley.edu False
yasho123@berkeley.edu False
awangs@berkeley.edu True
ishaan.mishra@berkeley.edu False
asap7772@berkeley.edu False
michellemboulos@berkeley.edu False
shilpasath@berkeley.edu False
jason.bustani@berkeley.edu False
andersontsai@berkeley.edu False
ajjiang@berkeley.edu False
lizvnguyen@gmail.com True
jyang8@berkeley.edu False
ahujaabhik@berkeley.edu False
schristopher@berkeley.edu False
shefalibhatia@berkeley.edu False
candyxu@berkeley.edu False
white_daniell@berkeley.edu False
kylelam@berkeley.edu False
isadoracw@berkeley.edu True
james.j.jiao@gmail.com True
stxiao@berkeley.edu False
hiaprillee@gmail.com True
ethanbmehta@berkeley.edu False
yujiewang@berkeley.edu False
hzpham2016@gmail.com True
johnianrso@berkeley.edu False
amywang98@berkeley.edu False
itoyuki@berkeley.edu False
sbanka@berkeley.edu False
kennethlien@berkeley.edu False
wang

yuweic@berkeley.edu False
kellyvtrinh@berkeley.edu False
2cyl@berkeley.edu False
anzhan@berkeley.edu False
trishasanghal@berkeley.edu False
meiqisun@berkeley.edu False
that.jonathan.liu@gmail.com True
john.k7@berkeley.edu False
itsanishkar@gmail.com True
nehav@berkeley.edu False
jonathanyun@berkeley.edu False
howardm12138@berkeley.edu False
wlouis23@berkeley.edu False
hubi@berkeley.edu False
wilsonxu123@berkeley.edu False
htce977@gmail.com True
strifale@berkeley.edu False
kkhus5@berkeley.edu False
arda.akman@berkeley.edu False
donnakim@berkeley.edu False
elaineqian@berkeley.edu False
jyxzhang@berkeley.edu False
shridhay@gmail.com True
ethanqiu@berkeley.edu False
vtneang@berkeley.edu False
vihan@berkeley.edu False
jhyoon03@berkeley.edu False
clairelee8@berkeley.edu False
amdickson@berkeley.edu False
alvinz@berkeley.edu True
jiarui.shan@berkeley.edu False
lcguo@berkeley.edu False
zbrian2040@berkeley.edu False
rajamanisarvesh@berkeley.edu False
sheesking13@gmail.com True
ecnahnes@gmail.co

,username,email,status,fullname,in_directory
2,ieeeucb,ieeeucb@gmail.com,Primary Owner,IEEE Account,0
3,kevinma,kevinma.sd@berkeley.edu,Member,Kevin Ma,0
13,daminig,daminig@berkeley.edu,Member,Damini Grover,0
20,brent,brentyi@berkeley.edu,Admin,Brent Yi,0
24,billz12oz,william_zhao@berkeley.edu,Member,Billy Zhao,0
...,...,...,...,...,...
777,eustyn_trinh,eustyn_trinh@berkeley.edu,Member,Waterstar1,1
779,richard95lee,richard95lee@berkeley.edu,Member,Richard Lee,1
780,scheduler,botuser-T1JAA6H2N-B01EPJZ3EHH@slack-bots.com,Bot,Message Scheduler,0
781,xr,xr@berkeley.edu,Member,Ray Xi,1


In [14]:
# filtering out slack members that exist in the cal directory
graduated = member_activity['in_directory'] == 0
graduated_members = member_activity[graduated]
graduated_members.to_csv('graduated_members.csv')

graduated_members

,username,email,status,fullname,in_directory
2,ieeeucb,ieeeucb@gmail.com,Primary Owner,IEEE Account,0
3,kevinma,kevinma.sd@berkeley.edu,Member,Kevin Ma,0
13,daminig,daminig@berkeley.edu,Member,Damini Grover,0
20,brent,brentyi@berkeley.edu,Admin,Brent Yi,0
24,billz12oz,william_zhao@berkeley.edu,Member,Billy Zhao,0
...,...,...,...,...,...
758,simplepoll,botuser-T1JAA6H2N-B01B43X7763@slack-bots.com,Bot,Simple Poll,0
763,github,appuser-T1JAA6H2N-A8GBNUWU8@slack-bots.com,Bot,GitHub,0
768,sam_wang_lisa1121,sam_wang_lisa1121@berkeley.edu,Member,Lisa Sam Wang,0
780,scheduler,botuser-T1JAA6H2N-B01EPJZ3EHH@slack-bots.com,Bot,Message Scheduler,0
